In [1]:
!pip install altair -q --user

# GEG EDA

<INTRO STUFF>

## Getting data on Evergiven timeframe

In [1]:
import json
#set params
BEGIN = "2021-03-01"
END = "2021-04-30"

HEIGHT = 400
WIDTH = 400

bq_params = {"BEGIN": BEGIN, "END": END}
bq_params = json.dumps(bq_params, indent=4, sort_keys=True, default=str)

In [8]:
%%bigquery april_events --params {"BEGIN": "2021-03-01", "END": "2021-04-30"}
WITH geg_data AS ((
    SELECT 
    groupId, 
    name, 
    a.entity as stringVal, 
    a.numMentions, 
    a.avgSalience, 
    eventTime, 
    polarity, 
    magnitude, 
    score 
    from (
    	SELECT polarity, 
        magnitude, 
        score, 
        FARM_FINGERPRINT(url) groupId, 
        CONCAT('Entity',entity.type) name, 
        FORMAT_TIMESTAMP("%Y-%m-%dT%X%Ez", date, "UTC") eventTime, 
    	entity.mid mid FROM `gdelt-bq.gdeltv2.geg_gcnlapi`, UNNEST(entities) entity WHERE entity.mid is not null 
       and lang='en' and DATE(date) >= @BEGIN and DATE(date) <= @END
  ) b JOIN (
     SELECT APPROX_TOP_COUNT(entities.name, 1)[OFFSET(0)].value entity, 
        entities.mid mid, sum(entities.numMentions) as numMentions, 
        avg(entities.avgSalience) as avgSalience
      FROM `gdelt-bq.gdeltv2.geg_gcnlapi`, 
     unnest(entities) entities where entities.mid is not null and lang='en' 
     and DATE(date) >= @BEGIN and DATE(date) <= @END group by entities.mid
  ) a USING(mid)))

select * from geg_data


Downloading: 100%|██████████| 70369292/70369292 [00:47<00:00, 1494310.07rows/s]


In [9]:
april_events

,groupId,name,stringVal,numMentions,avgSalience,eventTime,polarity,magnitude,score
0,7830976867991912493,EntityLOCATION,Gilbert Station Road,5,0.001257,2021-03-21T11:46:56+00:00,0.0,4.5,0.0
1,3100410252706223831,EntityORGANIZATION,UBS,8848,0.018045,2021-03-21T21:32:33+00:00,0.0,4.5,0.0
2,-2297817263941679196,EntityLOCATION,Los Angeles,205042,0.010001,2021-03-21T00:02:03+00:00,0.0,4.5,0.0
3,-5374852001300381844,EntityOTHER,Kolelas,1338,0.047422,2021-03-21T16:32:38+00:00,0.0,4.5,-0.3
4,-6754983596130560644,EntityOTHER,Kolelas,1338,0.047422,2021-03-21T17:01:45+00:00,0.0,4.5,-0.3
...,...,...,...,...,...,...,...,...,...
70369287,-1191005947168842791,EntityLOCATION,U.S.,4350763,0.016365,2021-03-09T01:18:31+00:00,0.0,7.5,-0.1
70369288,2970384450307155624,EntityLOCATION,U.S.,4350763,0.016365,2021-03-09T16:34:09+00:00,0.0,7.5,-0.4
70369289,4391024216151471084,EntityLOCATION,U.S.,4350763,0.016365,2021-03-09T07:18:18+00:00,0.0,8.0,0.0
70369290,-4657659261277881748,EntityLOCATION,U.S.,4350763,0.016365,2021-03-09T07:46:49+00:00,0.0,8.0,0.0


### Get a count of the entity values for this period

In [10]:
import altair as alt


def return_top_count_categories(data, group_field, depth):
    ### Get the top counts for stuff
    entity_count = data.groupby(group_field).count()
    top_n = entity_count.sort_values('groupId', ascending=False)[0:depth]
    other_count = entity_count.sort_values('groupId', ascending=False)['groupId'][depth+1::].sum()
    top_n = top_n.reset_index()
    if isinstance(group_field, list):
        top_n.append({'stringVal': 'Other', 'name': 'Other', 'eventTime': other_count}, ignore_index=True)
    else:
    	top_n = top_n.append({group_field: 'Other', 'eventTime': other_count}, ignore_index=True)
    return(top_n)                  
                         
def create_entity_count_charts(count_field, data, salience_filter=0, entity_term_filter=None, top_n=20):
    
    data = data[data['avgSalience'] >= salience_filter]
    if entity_term_filter:
        data=data[data[count_field].str.contains(entity_term_filter, case=False)]
    if top_n > 0:
    	entity_count = return_top_count_categories(data, count_field, top_n)
    else:
        entity_count = data.groupby(count_field).count()

    entity_count = entity_count.reset_index()
    entity_count = entity_count.sort_values('eventTime', ascending=False)
    entity_count['count'] = entity_count['eventTime']

    count_chart = alt.Chart(entity_count).mark_bar().encode(
        x={'field': f'{count_field}', 'type': 'ordinal', 'sort': 'y',},
        y={'field': 'count', 'type': 'quantitative'},
        tooltip=[count_field, 'count']
    ).properties(
        title=f"{count_field} Counts for March 2021",
        height=HEIGHT,
        width=WIDTH
    )
    
    return count_chart


# another one for the chart that uses colors for the categories

def create_entity_count_charts_color(data, salience_filter=0, entity_term_filter=None, top_n=20):
    
    data = data[data['avgSalience'] >= salience_filter]
    if entity_term_filter:
        data=data[data['stringVal'].str.contains(entity_term_filter, case=False)]
    if top_n > 0:
        entity_count = return_top_count_categories(data, ['stringVal', 'name'], top_n)
    else:
    	entity_count = data.groupby(['stringVal', 'name']).count()

    entity_count = entity_count.reset_index()
    entity_count = entity_count.sort_values('eventTime', ascending=False)
    entity_count['count'] = entity_count['eventTime']

    count_chart = alt.Chart(entity_count).mark_bar().encode(
        x={'field': 'stringVal', 'type':'ordinal', 'sort': 'y'},
        y={'field': 'count', 'type': 'quantitative'},
        tooltip=['name', 'stringVal', 'count'],
        color='name:N',
    ).properties(
        title=f"Combined Counts for March 2021",
        height=HEIGHT,
        width=WIDTH*2
    )
    
    return count_chart



In [11]:
## EverGiven is EntityOTHER - maybe we monitor that steam?

In [12]:
category_count_chart = create_entity_count_charts('name', april_events, salience_filter=0, entity_term_filter=None, top_n=100)
entity_count_chart = create_entity_count_charts('stringVal', april_events, salience_filter=0, entity_term_filter=None, top_n=50)
top_chart = create_entity_count_charts_color(april_events, salience_filter=0, entity_term_filter=None, top_n=50)
alt.data_transformers.disable_max_rows()

alt.vconcat(
    top_chart,
    alt.hconcat(category_count_chart, entity_count_chart),
)
    

alt.VConcatChart(...)

## Timeseries Analysis

In [ ]:
april_events[april_events.stringVal == 'Ever Given']

In [13]:
import datetime

import pandas as pd
def create_ts_chart_data(data, filter_val=["Ever Given", "Beverly Hills", "Donald Trump"]):
    data = data[data.stringVal.isin(filter_val)]
    data['eventTime'] = pd.to_datetime(data['eventTime']).dt.date
    avg_pol = data.groupby(['eventTime', 'stringVal'])['polarity'].mean()
    avg_mag = data.groupby(['eventTime', 'stringVal'])['magnitude'].mean()
    avg_score = data.groupby(['eventTime', 'stringVal'])['score'].mean()
    count = data.groupby(['eventTime', 'stringVal'])['numMentions'].sum()
    return pd.DataFrame([count, avg_pol, avg_mag, avg_score]).transpose().reset_index()


ts_data = create_ts_chart_data(april_events, filter_val=None)


ts_data['eventTime'] = ts_data['eventTime'].astype('datetime64[ns]')
ts_data

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [NoneType]

In [ ]:
alt.data_transformers.disable_max_rows()

avg_magnitude = alt.Chart(ts_data).mark_line().encode(
    x={"field": "eventTime", 'type': 'temporal'},
    y={'field': 'magnitude', 'type': 'quantitative'},
    tooltip=['eventTime','magnitude', 'stringVal'],
    color='stringVal:N',
).properties(
        title=f"Average Magnitude Score by Entity",
        height=HEIGHT,
        width=WIDTH
    )

num_mentions = alt.Chart(ts_data).mark_line().encode(
    x={"field": "eventTime", 'type': 'temporal'},
    y={'field': 'numMentions', 'type': 'quantitative'},
    tooltip=['eventTime','numMentions', 'stringVal'],
    color='stringVal:N',
).properties(
        title=f"Number of Mentions by Entity",
        height=HEIGHT,
        width=WIDTH
    )

avg_polarity = alt.Chart(ts_data).mark_line().encode(
    x={"field": "eventTime", 'type': 'temporal'},
    y={'field': 'polarity', 'type': 'quantitative'},
    tooltip=['eventTime','polarity', 'stringVal'],
    color='stringVal:N',
).properties(
        title=f"Average Polarity by Entity",
        height=HEIGHT,
        width=WIDTH
    )

avg_score = alt.Chart(ts_data).mark_line().encode(
    x={"field": "eventTime", 'type': 'temporal'},
    y={'field': 'score', 'type': 'quantitative'},
    tooltip=['eventTime','score', 'stringVal'],
    color='stringVal:N',
).properties(
        title=f"Avgerage Score by Entity",
        height=HEIGHT,
        width=WIDTH
    )

alt.vconcat(
    alt.hconcat(num_mentions, avg_magnitude),
    alt.hconcat(avg_polarity, avg_score),
)

In [144]:
ts_data.eventTime

0     2021-03-01
1     2021-03-01
2     2021-03-02
3     2021-03-02
4     2021-03-03
         ...    
155   2021-04-29
156   2021-04-29
157   2021-04-30
158   2021-04-30
159   2021-04-30
Name: eventTime, Length: 160, dtype: datetime64[ns]

In [145]:
alt.Chart(ts_data).mark_rect().encode(
    alt.X('hoursminutes(eventTime):O', title='hour of day'),
    alt.Y('monthdate(eventTime):O', title='date'),
    alt.Color('name:Q', title='temperature (F)')
)

alt.Chart(...)

In [41]:
w=april_events[april_events['stringVal'].lower == "tom petty"]
w

AttributeError: 'Series' object has no attribute 'lower'

In [63]:
zz=april_events[april_events['stringVal'].str.contains("z")]

In [64]:
zz

,groupId,name,stringVal,numMentions,avgSalience,eventTime,polarity,magnitude,score
31,-3105079244623794583,EntityPERSON,Queen Elizabeth,6.0,0.880214,2021-03-01T07:16:47+00:00,0.0,2.0,0.2
33,6107597291347116114,EntityPERSON,Queen Elizabeth,6.0,0.880214,2021-03-01T14:33:18+00:00,0.0,2.0,0.2
35,-1480449247246516428,EntityPERSON,Queen Elizabeth,6.0,0.880214,2021-03-01T03:32:30+00:00,0.0,2.0,0.2
37,8228111562446977735,EntityPERSON,Queen Elizabeth,6.0,0.880214,2021-03-01T12:32:36+00:00,0.0,2.0,0.2
39,-5462915138951645714,EntityPERSON,Queen Elizabeth,6.0,0.880214,2021-03-01T08:02:55+00:00,0.0,2.0,0.2
...,...,...,...,...,...,...,...,...,...
4328367,-5085596850111108760,EntityORGANIZATION,World Health Organization,4.0,0.851978,2021-03-18T08:19:06+00:00,0.0,1.8,0.0
4328368,-35215210535195339,EntityORGANIZATION,World Health Organization,4.0,0.851978,2021-03-18T01:17:51+00:00,0.0,1.8,-0.2
4328369,2322684724690910539,EntityORGANIZATION,World Health Organization,4.0,0.851978,2021-03-18T22:32:15+00:00,0.0,1.8,0.0
4328370,8993312145740526654,EntityORGANIZATION,World Health Organization,4.0,0.851978,2021-03-18T17:01:52+00:00,0.0,1.8,-0.3


In [112]:
### Get the top counts for stuff

entity_count = zz.groupby("stringVal").count()
top_n = entity_count.sort_values('stringVal', ascending=False)[0:20]
other_count = entity_count.sort_values('groupId', ascending=False)['groupId'][21::].sum()
top_n = top_n.reset_index()
top_n = top_n.append({'stringVal': 'Other', 'groupId': other_count}, ignore_index=True)

In [145]:
#debug
data = zz
### Get the top counts for stuff
entity_count = data.groupby('stringVal').count()
top_n = entity_count.sort_values('groupId', ascending=False)[0:20]
other_count = entity_count.sort_values('groupId', ascending=False)['groupId'][20+1::].sum()
top_n = top_n.reset_index()

In [147]:
top_n

,stringVal,groupId,name,numMentions,avgSalience,eventTime,polarity,magnitude,score
0,World Health Organization,43845,43845,43845,43845,43845,43845,43845,43845
1,Amazon,38094,38094,38094,38094,38094,38094,38094,38094
2,Queen Elizabeth,12228,12228,12228,12228,12228,12228,12228,12228
3,Ted Cruz,5963,5963,5963,5963,5963,5963,5963,5963
4,Southern Azerbaijan,4478,4478,4478,4478,4478,4478,4478,4478
5,Jennifer Lopez,3578,3578,3578,3578,3578,3578,3578,3578
6,Alexandria Ocasio-Cortez,3462,3462,3462,3462,3462,3462,3462,3462
7,Riz Ahmed,2720,2720,2720,2720,2720,2720,2720,2720
8,Nazanin Zaghari-Ratcliffe,1808,1808,1808,1808,1808,1808,1808,1808
9,Arnold Schwarzenegger,1690,1690,1690,1690,1690,1690,1690,1690


AttributeError: 'DataFrame' object has no attribute 'type'